In [1]:
import pandas as pd
import numpy as np
from nltk.tokenize import word_tokenize
from nltk import pos_tag
from nltk.corpus import stopwords
from nltk.stem import WordNetLemmatizer
from sklearn.preprocessing import LabelEncoder
from collections import defaultdict
from nltk.corpus import wordnet as wn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import model_selection, naive_bayes, svm
from sklearn.metrics import accuracy_score

In [2]:
np.random.seed(500)

In [3]:
Corpus = pd.read_csv(r"/home/theredlad/Desktop/CV/topic_Classification_NV_SVM/corpus.csv",encoding='latin-1')

In [4]:
#Remove blank rows if any.
Corpus['text'].dropna(inplace=True)

In [5]:
#Tokenization : In this each entry in the corpus will be broken into set of words
Corpus['text']= [word_tokenize(entry) for entry in Corpus['text']]

In [6]:
# Step - d : Remove Stop words, Non-Numeric and perfom Word Stemming/Lemmenting.
# WordNetLemmatizer requires Pos tags to understand if the word is noun or verb or adjective etc. 
#By default it is set to Noun
tag_map = defaultdict(lambda : wn.NOUN)
tag_map['J'] = wn.ADJ
tag_map['V'] = wn.VERB
tag_map['R'] = wn.ADV

In [7]:
for index,entry in enumerate(Corpus['text']):
    # Declaring Empty List to store the words that follow the rules for this step
    Final_words = []
    # Initializing WordNetLemmatizer()
    word_Lemmatized = WordNetLemmatizer()
    # pos_tag function below will provide the 'tag' i.e if the word is Noun(N) or Verb(V) or something else.
    for word, tag in pos_tag(entry):
        # Below condition is to check for Stop words and consider only alphabets
        if word not in stopwords.words('english') and word.isalpha():
            word_Final = word_Lemmatized.lemmatize(word,tag_map[tag[0]])
            Final_words.append(word_Final)
    # The final processed set of words for each iteration will be stored in 'text_final'
    Corpus.loc[index,'text_final'] = str(Final_words)

In [8]:
Train_X, Test_X, Train_Y, Test_Y = model_selection.train_test_split(Corpus['text_final'],Corpus['label'],test_size=0.3)

In [9]:
#Label encode the target variable — 
#This is done to transform Categorical data of string type in the data set into numerical values 
#which the model can understand.

Encoder = LabelEncoder()
Train_Y = Encoder.fit_transform(Train_Y)
Test_Y = Encoder.fit_transform(Test_Y)

In [10]:
Tfidf_vect = TfidfVectorizer(max_features=5000)
Tfidf_vect.fit(Corpus['text_final'])

Train_X_Tfidf = Tfidf_vect.transform(Train_X)
Test_X_Tfidf = Tfidf_vect.transform(Test_X)

In [11]:
print(Tfidf_vect.vocabulary_)

{'even': 1508, 'this': 4442, 'sound': 4103, 'track': 4539, 'beautiful': 387, 'it': 2364, 'paint': 3157, 'mind': 2832, 'well': 4848, 'would': 4950, 'recomend': 3582, 'people': 3222, 'hate': 2044, 'video': 4752, 'game': 1854, 'music': 2919, 'play': 3297, 'cross': 1028, 'ever': 1512, 'best': 427, 'back': 332, 'away': 322, 'crude': 1031, 'take': 4343, 'fresh': 1809, 'step': 4191, 'guitar': 1997, 'orchestra': 3096, 'impress': 2228, 'anyone': 204, 'care': 647, 'listen': 2589, 'the': 4414, 'soundtrack': 4105, 'anything': 205, 'read': 3547, 'lot': 2637, 'review': 3708, 'say': 3824, 'figure': 1682, 'write': 4955, 'disagree': 1218, 'bit': 449, 'ultimate': 4618, 'masterpiece': 2736, 'timeless': 4481, 'year': 4974, 'beauty': 389, 'simply': 3997, 'refuse': 3607, 'price': 3399, 'tag': 4342, 'pretty': 3394, 'must': 2925, 'go': 1922, 'buy': 599, 'cd': 680, 'much': 2910, 'money': 2868, 'one': 3074, 'feel': 1658, 'worth': 4946, 'every': 1514, 'penny': 3221, 'amazing': 153, 'favorite': 1646, 'time': 4480

In [12]:
# fit the training dataset on the NB classifier
Naive = naive_bayes.MultinomialNB()
Naive.fit(Train_X_Tfidf,Train_Y)# predict the labels on validation dataset

MultinomialNB()

In [13]:
predictions_NB = Naive.predict(Test_X_Tfidf)# Use accuracy_score function to get the accuracy

In [14]:
# Classifier - Algorithm - SVM
# fit the training dataset on the classifier
SVM = svm.SVC(C=1.0, kernel='linear', degree=3, gamma='auto')
SVM.fit(Train_X_Tfidf,Train_Y)

SVC(gamma='auto', kernel='linear')

In [15]:
# predict the labels on validation dataset
predictions_SVM = SVM.predict(Test_X_Tfidf)

In [16]:
# Use accuracy_score function to get the accuracy
print("Naive Bayes Accuracy Score -> ",accuracy_score(predictions_NB, Test_Y)*100)
print("SVM Accuracy Score -> ",accuracy_score(predictions_SVM, Test_Y)*100)

Naive Bayes Accuracy Score ->  83.7
SVM Accuracy Score ->  85.46666666666667
